In [1]:
# FUNCTIONS

# for MIR-QBSH query
def read_semitone_from_MIR_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    for file in os.listdir(dir_query):
        if file.endswith(".csv"):
            list_query_name.append(file.replace(".csv",""))
            file_path = os.path.join(dir_query, file)
            fields = ["semitone"]
            temp_df = pd.read_csv(file_path, usecols=fields)
            df_query.append(temp_df.to_numpy().flatten())
            truth.append(file.split("-")[2])

    return list_query_name, df_query, truth

# for MIR-QBSH manually labelled query
def read_semitone_from_MIR_manual_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    for file in os.listdir(dir_query):
        if file.endswith(".pv"):
            list_query_name.append(file.replace(".pv",""))
            file_path = os.path.join(dir_query, file)
            temp_df = pd.read_csv(file_path)
            temp_df = temp_df.to_numpy().flatten()
            round_df = []
            for i in temp_df:
                if i!=0:
                    round_df.append(round(i))
            df_query.append(round_df)
            truth.append(file.replace(".pv","").split("-")[2])

    return list_query_name, df_query, truth

# for IOACAS_QBH query
def read_semitone_from_IOACAS_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    truth_file = os.path.join(dir_query, "query_truth.list")

    # read csv as string type need converters
    truth_df = pd.read_csv(truth_file, header=None, sep="\t", converters={i: str for i in range(100)})

    all_truth = truth_df[1].values.astype(str).flatten()
    # print("all truth", all_truth)
    all_wav = []

    for wav in truth_df[0]:
        all_wav.append(wav.split("\\")[1].replace(".wav",""))

    for file in os.listdir(dir_query):
        if file.endswith(".csv"):
            list_query_name.append(file.replace(".csv",""))
            file_path = os.path.join(dir_query, file)
            fields = ["semitone"]
            temp_df = pd.read_csv(file_path, usecols=fields)
            df_query.append(temp_df.to_numpy().flatten())
            # index_truth = all_wav.index("002_010")
            index_truth = all_wav.index(file.split("-")[0])
            truth.append(str(all_truth[index_truth]))
    # print("one truth", truth)

    return list_query_name, df_query, truth

# for MIR-QBSH and IOACAS-QBH database
def read_note_from_midi(dir_midi):
    pickled_semitone = "pickled_"+dir_midi.replace(folder_data+"\\","")
    dir_pickle = os.path.join(folder_pickle,pickled_semitone)

    if os.path.exists(dir_pickle):
        open_file = open(dir_pickle, "rb")
        list_dir_midi, df_midi = pickle.load(open_file)
        open_file.close()
    else:
        open_file = open(dir_pickle, "wb")
        list_dir_midi = []
        df_midi = []

        for file in os.listdir(dir_midi):
            if file.endswith(".csv"):
                list_dir_midi.append(file.replace(".mid.csv",""))
                file_path = os.path.join(dir_midi, file)
                fields = ["note_index"]
                temp_df = pd.read_csv(file_path, usecols=fields)
                df_midi.append(temp_df.to_numpy().flatten())
        pickle.dump([list_dir_midi, df_midi], open_file)
        open_file.close()

    return list_dir_midi, df_midi

# calculate relative distance
def calc_rel_dis(df):
    res = []
    length = len(df)
    for i in range(length-1):
        dis = float(df[i+1] - df[i])
        res.append(dis)
    return res

# remove consecutive distance in list of relative distance 
def remove_consecutive(list):
    i = 0
    while i < len(list)-1:
        if list[i] == list[i+1]:
            del list[i]
        else:
            i = i+1
    return list

# arrange index according to its similarity, more similar less 
def get_all_rank(ls):
    res = []
    for i in ls:
        temp = ls.index(max(ls))
        res.append(list_dir_midi[temp])
        ls[temp] = 0
    return res

# get rank from ground truth
def get_rank(rank, truth):
    return rank.index(truth)+1

# get top ten rank
def get_top_ten(rank):
    return rank[:10]

# get inverted index from all midi
def get_inverted_index_midi(dis_midi, list_dir_midi):
    # create inverted index for 2-grams, 3-grams, 4-grams in all midi
    import hashedindex
    RP4G = hashedindex.HashedIndex()
    RP3G = hashedindex.HashedIndex()
    RP2G = hashedindex.HashedIndex()

    # example:
    # midi relative distance : +1 +1 +4
    # 2-grams are +1 and +4
    # 3-grams are +1 +1 and +1 +3
    # 4-grams is +1 +1 +4

    # print(dis_midi[0])

    # inserting 2-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for note in dis_midi[midiNumber]:
            RP2G.add_term_occurrence(note, list_dir_midi[midiNumber])

    # test 2-grams
    # RP2G.get_documents((2.0))

    # inserting 3-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for noteNumber in range(len(dis_midi[midiNumber])-1):
            term = (dis_midi[midiNumber][noteNumber], dis_midi[midiNumber][noteNumber+1])
            RP3G.add_term_occurrence(term, list_dir_midi[midiNumber])

    # test 3-grams
    # RP3G.get_documents((2.0, 0.0))

    # inserting 4-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for noteNumber in range(len(dis_midi[midiNumber])-2):
            term = (dis_midi[midiNumber][noteNumber], 
            dis_midi[midiNumber][noteNumber+1], dis_midi[midiNumber][noteNumber+2])
            
            RP4G.add_term_occurrence(term, list_dir_midi[midiNumber])

    # test 4-grams
    # RP4G.get_documents((2.0, 0.0, -2.0))

    return RP4G, RP3G, RP2G

# get lists containing 4-grams, 3-grams, and 2-grams from each query
def get_ngrams_query(dis_query):
    RP4G = []
    RP3G = []
    RP2G = []

    # inserting list of 4-grams
    for noteNumber in range(len(dis_query)-2):
        term = (dis_query[noteNumber], dis_query[noteNumber+1], dis_query[noteNumber+2])
        RP4G.append(term)

    # inserting list of 3-grams
    for noteNumber in range(len(dis_query)-1):
        term = (dis_query[noteNumber], dis_query[noteNumber+1])
        RP3G.append(term)

    # inserting list of 2-grams
    RP2G = dis_query
    
    return RP4G, RP3G, RP2G

from collections import Counter

# get Counter containing number of query appearances in inverted index
def get_counter_grams(index, query):
    res = Counter()
    for grams in query:
        try:
            res += index.get_documents(grams)
        except IndexError as error:
            pass
    return res

# get rank from 
def get_rank_from_counter_grams(res_grams, truth):
    res = []
    for i in res_grams.most_common():
        res.append(i[0])
    try:
        rank = get_rank(res, truth)
    except ValueError as error:
        rank = sys.maxsize
    return rank

# get rank with relative pitch 4-grams (RP4G), 3-grams (RP3G) and 2-grams (RP2G)
def get_rank_with_rpg(dis_query, truth, index_midi):
    index4G, index3G, index2G = index_midi
    query4G, query3G, query2G = get_ngrams_query(dis_query)

    # 4-grams
    res_4grams = get_counter_grams(index4G, query4G)

    # 3-grams
    res_3grams = get_counter_grams(index3G, query3G)

    # 2-grams
    res_2grams = get_counter_grams(index2G, query2G)

    res_rank = []

    rank = get_rank_from_counter_grams(res_4grams, truth)
    res_rank.append(rank)

    rank = get_rank_from_counter_grams(res_3grams, truth)
    res_rank.append(rank)

    rank = get_rank_from_counter_grams(res_2grams, truth)
    res_rank.append(rank)

    return res_rank

# get rank with Mode Normalised Frequency using edit distance method
def get_rank_with_mnf(df_query, truth, midi_MNF, cons_10):
    # ref = dis_midi
    # hyp = dis_query

    hyp = convert_df_to_MNF(df_query, cons_10)
    
    list_ratio = []
    for ref in midi_MNF:
        sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
        list_ratio.append(sm.ratio())
    
    rank = get_all_rank(list_ratio)
    rankTruth = get_rank(rank, truth)

    # show all rank
    # print("All rank from MNF:", rank)

    # show top ten result
    # print("Top ten from MNF:", get_top_ten(rank))

#     try:
#         rankTruth = get_rank(rank, truth)
#     except ValueError as error:
#         rankTruth = len(dis_midi)
    # topTen = get_top_ten(rank)

    return rankTruth

# get best rank from both nGrams and edit distance method
def get_rank_with_unified_algorithm(dis_query, df_query, truth, index_midi, midi_MNF):
    res = get_rank_with_rpg(dis_query, truth, index_midi)

    try:
        res.append(get_rank_with_mnf(df_query, truth, midi_MNF, cons_10=True))
    except ValueError as error:
        pass
    except IndexError as error:
        pass

    bestRank = min(res)
    return bestRank

# convert one dataframe containing semitone to Mode Normalised Frequency (MNF)
def convert_df_to_MNF(df,cons_10,remove_cons=False):
    from collections import Counter
    res = []
    data = Counter(df)

    data = data.most_common()
    mode = data[0][0]
    adder = int(78-mode) # mode marked as char 'N' which is 78 in ascii

    for i in df:
        res.append(chr(int(i+adder)))

    if cons_10==True:
        res = get_only_10_consecutive_notes(res)

    if remove_cons==True:
        res = remove_consecutive(res)

    return res

from itertools import groupby
def get_only_10_consecutive_notes(ls):
    res = []
    n = 10
    for k, v in groupby(ls):
        value = list(v)
        if len(value) >= n:
            res.append(value[0])
    return res

# count MRR from a list of rank from some queries
def count_MRR(list_rank):
    mrr = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            mrr+=1/rank
            counter+=1
    return round(mrr/counter,2), counter

# count top 1 hit ratios from list rank
def count_top_1_ratio(list_rank):
    count_top1 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank == 1:
                count_top1+=1
            counter+=1
    return round(count_top1/counter,2), counter

# count top 3 hit ratios from list rank
def count_top_3_ratio(list_rank):
    count_top3 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 3:
                count_top3+=1
            counter+=1
    return round(count_top3/counter,2), counter

# count top 5 hit ratios from list rank
def count_top_5_ratio(list_rank):
    count_top5 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 5:
                count_top5+=1
            counter+=1
    return round(count_top5/counter,2), counter

# count top 10 hit ratios from list rank
def count_top_10_ratio(list_rank):
    count_top10 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 10:
                count_top10+=1
            counter+=1
    return round(count_top10/counter,2), counter

def print_result_to_file(process_name):
    from datetime import datetime

    dateTimeObj = str(datetime.now())+".txt"
    dateTimeObj = dateTimeObj.replace(":","-")
    filename = os.path.join(folder_hasil, process_name + ' ' + dateTimeObj)
    with open(filename, 'w') as f:
        f.write(cap.stdout)

def process_query():
    # calculate relative distance in all query and midi
    dis_query, dis_midi = get_rel_dis_query_midi(df_query, df_midi)

    index_midi = get_inverted_index_midi(dis_midi, list_dir_midi)

    midi_MNF = []
    for i in df_midi:
        midi_MNF.append(convert_df_to_MNF(i, cons_10=False, remove_cons=False))

    list_rank = []
    list_time = []

    process_name = dir_query.replace(folder_data+"\\","")+" with "+dir_midi.replace(folder_data+"\\","")
    print("Start process", process_name)

    for i in range(len(df_query)):
        print("Processing...",i+1,"/",len(df_query))
        
        start_time = time.time()

        rankTruth = get_rank_with_unified_algorithm(dis_query[i], df_query[i], truth[i], index_midi, midi_MNF)
        
        list_time.append(round(time.time() - start_time, 3))

        list_rank.append(rankTruth)

        # show ground truth from a query
        # print("query",list_query_name[i],"truth is",truth[i])

        print("query",list_query_name[i],"truth is on rank",rankTruth)

    print("Finish process", process_name)
    
    try:
        mrr, counter = count_MRR(list_rank)
        print("MRR:", mrr, "from", counter, "queries")

        hit_ratio, counter = count_top_1_ratio(list_rank)
        print("Top 1 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_3_ratio(list_rank)
        print("Top 3 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_5_ratio(list_rank)
        print("Top 5 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_10_ratio(list_rank)
        print("Top 10 ratio:", hit_ratio, "from", counter, "queries" )
        
        print("Avg time:", round(sum(list_time)/len(list_time), 3))
    
    except ZeroDivisionError:
        print("No result to show")
        print("list rank", list_rank)
        print("list time", list_time)

    return process_name

# convert pitch to semitone
def convert_f0_to_semitone(f0):
    import math

    res = []
    for freq in f0:
        res.append(round(12*math.log((freq/440),2)+69))
    return res

# extract pitch and semitone with parselmouth from wav to csv
def extract_pitch_with_parselmouth_to_csv(folder, dir_out):
    import parselmouth
    import numpy as np
    import os
    import pandas as pd

    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(dir_wav_query, file)

            snd = parselmouth.Sound(file_path)

            pitch = snd.to_pitch()
            pitch_values = pitch.selected_array['frequency']

            time = []
            f0 = []

            for i in range(len(pitch_values)):
                if pitch_values[i]!=0:
                    f0.append(pitch_values[i].round(decimals=3))
                    time.append(pitch.xs()[i].round(decimals=3))

            file = file_path.split("\\")
            file_csv = os.path.join(dir_out, file[2] + "-parselmouth.csv")
            file_csv = file_csv.replace(".wav","")

            semitone = convert_f0_to_semitone(f0)

            pd.DataFrame({'time':time, 'semitone':semitone, 'f0':f0}).to_csv(file_csv, index=False)

# extract pitch and semitone with DNN-LSTM from wav to csv
def extract_pitch_with_DNN_LSTM_to_csv(folder, dir_out):
    import shutil
    
    folder_project_DNN_LSTM = "Lab_MelExt"

    dir_temp_in = os.path.join(folder_project_DNN_LSTM, "temp_query")
    if not os.path.exists(dir_temp_in):
        os.mkdir(dir_temp_in)

    for i in os.listdir(folder):
        source = os.path.join(folder, i)
        destination = os.path.join(dir_temp_in, i)
        shutil.copy(source, destination)

    print("Start melody extraction with DNN-LSTM")
    cwd = os.getcwd()
    os.chdir(folder_project_DNN_LSTM)
    cmd_line = "python predict.py temp_query temp_pitch"
    os.system(cmd_line)
    os.chdir(cwd)
    print("Finish melody extraction with DNN-LSTM")

    dir_temp_query = os.path.join(folder_project_DNN_LSTM, "temp_pitch")

    for file in os.listdir(dir_temp_query):
        if file.endswith(".csv"):
            filePath = os.path.join(dir_temp_query, file)
            df = pd.read_csv(filePath, sep="\t", header=None)

            time = []
            f0 = []

            for i in range(len(df[1])):
                if df[1][i]!=0:
                    time.append(df[0][i].round(decimals=3))
                    f0.append(df[1][i].round(decimals=3))
                    
            semitone = convert_f0_to_semitone(f0)

            file_csv = os.path.join(dir_out, file)

            pd.DataFrame({'time':time, 'semitone':semitone, 'f0':f0}).to_csv(file_csv, index=False)
    
    shutil.rmtree(dir_temp_in)
    shutil.rmtree(dir_temp_query)

def extract_pitch_to_csv(extractor, folder, dir_out):
    if extractor == "parselmouth":
        extract_pitch_with_parselmouth_to_csv(folder, dir_out)
    elif extractor == "DNN-LSTM":
        extract_pitch_with_DNN_LSTM_to_csv(folder, dir_out)

# choose midi based on dir_query
def get_midi_dir(dir_query):
    if "48" in dir_query:
        dir_midi = os.path.join(folder_data, "Database midi IOACAS-QBH 48 MIDI")
    elif "IOACAS" in dir_query:
        dir_midi = os.path.join(folder_data, "Database midi IOACAS-QBH")
    else:
        dir_midi = os.path.join(folder_data, "Database midi MIR-QBSH")
    
    return dir_midi

# read and insert semitone from query to list of dataframe
def get_semitone_list(dir_query):
    pickled_semitone = "pickled_"+dir_query.replace(folder_data+"\\","")
    dir_pickle = os.path.join(folder_pickle,pickled_semitone)

    if os.path.exists(dir_pickle):
        open_file = open(dir_pickle, "rb")
        list_query_name, df_query, truth = pickle.load(open_file)
        open_file.close()
    else:
        open_file = open(dir_pickle, "wb")
        if "manual" in dir_query:
            list_query_name, df_query, truth = read_semitone_from_MIR_manual_query(dir_query)
        elif "IOACAS" in dir_query:
            list_query_name, df_query, truth = read_semitone_from_IOACAS_query(dir_query)
        else:
            list_query_name, df_query, truth = read_semitone_from_MIR_query(dir_query)
        pickle.dump([list_query_name, df_query, truth], open_file)
        open_file.close()

    return list_query_name, df_query, truth

# calculate relative distance in all query and midi
def get_rel_dis_query_midi(df_query, df_midi):
    # calculate relative distance in all query
    dis_query = []
    for query in df_query:
        dis_query.append(calc_rel_dis(query))

    # calculate relative distance in all midi
    dis_midi = []
    for midi in df_midi:
        dis_midi.append(calc_rel_dis(midi))

    return dis_query, dis_midi


In [2]:
# %%capture cap --no-stderr

# read midi and query data
# insert it into dataframe

import os
import pandas as pd
import edit_distance
import time
import sys
import pickle

folder_data = "Data query dan midi"
folder_hasil = "Hasil eksperimen"
folder_pickle = "pickle"

if not os.path.exists(folder_hasil):
    os.mkdir(folder_hasil)

if not os.path.exists(folder_pickle):
    os.mkdir(folder_pickle)

# test query
dir_query = os.path.join(folder_data, "Test query MIR-QBSH_parselmouth")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_manual label")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_parselmouth")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_parselmouth_48_MIDI")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_DNN-LSTM_48_MIDI")

# all query
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_parselmouth")
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_manual label")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_parselmouth")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_parselmouth_48_MIDI")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_DNN-LSTM_48_MIDI")

# choose midi based on dir_query
dir_midi = get_midi_dir(dir_query)

# read and insert semitone from query to list of dataframe
list_query_name, df_query, truth = get_semitone_list(dir_query)

# read and insert note_index or semitone from midi to list of dataframe
list_dir_midi, df_midi = read_note_from_midi(dir_midi)

# test match with query data
process_name = process_query()

# place on different cell
# print_result_to_file(process_name)

In [16]:
# %%capture cap --no-stderr

# FOR DEMO PURPOSES
# COMBINED ALL PROCESS FROM PITCH EXTRACTION TO MATCHING
# PARSELMOUTH AND DNN-LSTM PITCH EXTRACTION

import os
import pandas as pd
import edit_distance
import time
import sys
import shutil

folder_data = "Data query dan midi"
folder_hasil = "Hasil eksperimen"
# folder_wav = "Test wav query MIR-QBSH"
# folder_wav = "Test wav query IOACAS_QBH"
folder_wav = "Test wav query IOACAS_QBH_48_MIDI"

if not os.path.exists(folder_hasil):
    os.mkdir(folder_hasil)

# start pitch extraction

extractor = "parselmouth"
# extractor = "DNN-LSTM"

dir_wav_query = os.path.join(folder_data, folder_wav)

dir_query = dir_wav_query+"_"+extractor

if not os.path.exists(dir_query):
    os.mkdir(dir_query)

# copy IOACAS ground truth list file
if "IOACAS" in dir_wav_query:
    source = os.path.join(dir_wav_query, "query_truth.list")
    destination = os.path.join(dir_query, "query_truth.list")
    shutil.copy(source, destination)

extract_pitch_to_csv(extractor, dir_wav_query, dir_query)
    
# end pitch extraction

# start matching

# choose midi based on dir_query
dir_midi = get_midi_dir(dir_query)

# read and insert semitone from query to list of dataframe
list_query_name, df_query, truth = get_semitone_list(dir_query)

# read and insert note_index or semitone from midi to list of dataframe
list_dir_midi, df_midi = read_note_from_midi(dir_midi)

# test match with query data
process_name = process_query()

# place on different cell
# print_result_to_file(process_name)

# end matching

Start process Test wav query IOACAS_QBH_48_MIDI_parselmouth with Database midi IOACAS-QBH 48 MIDI
Processing... 1 / 3
query 001_003-parselmouth truth is on rank 3
Processing... 2 / 3
query 001_004-parselmouth truth is on rank 2
Processing... 3 / 3
query 001_008-parselmouth truth is on rank 3
Finish process Test wav query IOACAS_QBH_48_MIDI_parselmouth with Database midi IOACAS-QBH 48 MIDI
MRR: 0.39 from 3 queries
Top 3 ratio: 1.0 from 3 queries
Top 5 ratio: 1.0 from 3 queries
Top 10 ratio: 1.0 from 3 queries
Avg time: 5.344
